In [30]:
#import statements
from pyspark.sql import SparkSession

spark = SparkSession.builder.enableHiveSupport().appName('ReadWriteData').getOrCreate()
sc = spark.sparkContext

In [62]:
#check to see if thata is available in HDFS
!hdfs dfs -ls /user/akarshsahu/data/Netflix_Data

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
Found 11 items
-rw-r--r--   3 akarshsahu akarshsahu    1430649 2020-02-22 01:17 /user/akarshsahu/data/Netflix_Data/Title_Data_Final.csv
-rw-r--r--   3 akarshsahu akarshsahu  495032008 2020-02-19 17:39 /user/akarshsahu/data/Netflix_Data/combined_data_1.txt
-rw-r--r--   3 akarshsahu akarshsahu  555207357 2020-02-19 17:39 /user/akarshsahu/data/Netflix_Data/combined_data_2.txt
-rw-r--r--   3 akarshsahu akarshsahu  465157779 2020-02-19 17:39 /user/akarshsahu/data/Netflix_Data/combined_data_3.txt
-rw-r--r--   3 akarshsahu akarshsahu  552537802 2020-02-19 17:45 /user/akarshsahu/data/Netflix_Data/combined_data_4.txt
-rw-r--r--   3 akarshsahu akarshsahu 2599929944 2020-02-20 01:45 /user/akarshsahu/data/Netflix_Data/combined_data_final.csv
-rw-r--r--   3 akarshsahu akarshsahu     577547 2020-02-19 17:39 /user/akarshsahu/data/Netflix_Data/movie_titles.csv
drwxr-xr-x   - akarshsahu akarshsahu   

In [32]:
df = spark.read.csv("/user/akarshsahu/data/Netflix_Data/combined_data_final.csv", inferSchema=True, header=True)
title = spark.read.csv("/user/akarshsahu/data/Netflix_Data/Title_Data_Final.csv", inferSchema=True, header=True)

In [33]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Cust_Id: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Movie_Id: integer (nullable = true)



In [34]:
title.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Movie_Id: integer (nullable = true)
 |-- tconst: string (nullable = true)
 |-- Year: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- startYear: double (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: double (nullable = true)
 |-- Actor_0: string (nullable = true)
 |-- Actor_1: string (nullable = true)
 |-- Actor_2: string (nullable = true)
 |-- Actor_3: string (nullable = true)
 |-- Actor_4: string (nullable = true)
 |-- Actor_5: string (nullable = true)
 |-- Actor_6: string (nullable = true)
 |-- Actor_7: string (nullable = true)
 |-- Actor_8: string (nullable = true)
 |-- Actor_9: string (nullable = true)
 |-- Director_0: string (nullable = true)
 |-- Director_1: string (nullable = true)

In [35]:
df.show(1000)

+----+-------+------+--------+
| _c0|Cust_Id|Rating|Movie_Id|
+----+-------+------+--------+
|   1|1488844|   3.0|       1|
|   2| 822109|   5.0|       1|
|   3| 885013|   4.0|       1|
|   4|  30878|   4.0|       1|
|   5| 823519|   3.0|       1|
|   6| 893988|   3.0|       1|
|   7| 124105|   4.0|       1|
|   8|1248029|   3.0|       1|
|   9|1842128|   4.0|       1|
|  10|2238063|   3.0|       1|
|  11|1503895|   4.0|       1|
|  12|2207774|   5.0|       1|
|  13|2590061|   3.0|       1|
|  14|   2442|   3.0|       1|
|  15| 543865|   4.0|       1|
|  16|1209119|   4.0|       1|
|  17| 804919|   4.0|       1|
|  18|1086807|   3.0|       1|
|  19|1711859|   4.0|       1|
|  20| 372233|   5.0|       1|
|  21|1080361|   3.0|       1|
|  22|1245640|   3.0|       1|
|  23| 558634|   4.0|       1|
|  24|2165002|   4.0|       1|
|  25|1181550|   3.0|       1|
|  26|1227322|   4.0|       1|
|  27| 427928|   4.0|       1|
|  28| 814701|   5.0|       1|
|  29| 808731|   4.0|       1|
|  30| 6

In [36]:
title.show(3)

+---+--------+---------+------+--------------------+--------------------+---------+--------------+---------+-------+--------------------+-------------+--------+------------+--------------------+--------------+---------------+-------+-------+-------+-------+-------+-------+-----------------+----------+----------+----------+----------+----------+
|_c0|Movie_Id|   tconst|  Year|                Name|        primaryTitle|titleType|runtimeMinutes|startYear|isAdult|              genres|averageRating|numVotes|     Actor_0|             Actor_1|       Actor_2|        Actor_3|Actor_4|Actor_5|Actor_6|Actor_7|Actor_8|Actor_9|       Director_0|Director_1|Director_2|Director_3|Director_4|Director_5|
+---+--------+---------+------+--------------------+--------------------+---------+--------------+---------+-------+--------------------+-------------+--------+------------+--------------------+--------------+---------------+-------+-------+-------+-------+-------+-------+-----------------+----------+----

In [44]:
final_df = df.join(title, on = "Movie_Id")
final_df = final_df.drop('_c0')

In [45]:
final_df.show()

+--------+-------+------+---------+------+---------+------------+---------+--------------+---------+-------+-------------------+-------------+--------+-----------+--------------------+--------------+---------------+-------+-------+-------+-------+-------+-------+-------------+----------+----------+----------+----------+----------+
|Movie_Id|Cust_Id|Rating|   tconst|  Year|     Name|primaryTitle|titleType|runtimeMinutes|startYear|isAdult|             genres|averageRating|numVotes|    Actor_0|             Actor_1|       Actor_2|        Actor_3|Actor_4|Actor_5|Actor_6|Actor_7|Actor_8|Actor_9|   Director_0|Director_1|Director_2|Director_3|Director_4|Director_5|
+--------+-------+------+---------+------+---------+------------+---------+--------------+---------+-------+-------------------+-------------+--------+-----------+--------------------+--------------+---------------+-------+-------+-------+-------+-------+-------+-------------+----------+----------+----------+----------+----------+
|

In [46]:
from pyspark.sql.functions import col, countDistinct
final_df.agg(countDistinct(col("Movie_Id")).alias("count")).show()

+-----+
|count|
+-----+
| 7602|
+-----+



In [47]:
final_df.count()

79000155

In [61]:
import pandas as pd 
final_df_summ = final_df.describe().toPandas()

In [63]:
final_df_summ

summary           Movie_Id             Cust_Id              Rating  \
0   count           79000155            79000155            79000155   
1    mean  9120.794157517792  1322168.4876444482   3.573533608889755   
2  stddev  5148.096802532097   764548.8841645767  1.0637526570249025   
3     min                  3                   6                 1.0   
4     max              17770             2649429                 5.0   

      tconst                Year             Name     primaryTitle titleType  \
0   79000155            79000155         79000155         79000155  79000155   
1       None  1993.9761599075343         Infinity         Infinity      None   
2       None  12.246027864725871              NaN              NaN      None   
3  tt0003740              1914.0  'Round Midnight  'Round Midnight     movie   
4  tt8001886              2005.0             Zulu             Zulu     movie   

       runtimeMinutes       ...               Actor_6         Actor_7  \
0            79000155       ...                 77352           23987   
1  114.08969772466837       ...                  None            None   
2  20.941956305363153       ...                  None            None   
3                 100       ...              Ada Choi    Amnon Meskin   
4                  \N       ...         Yun-seong Lee  Yeung-Ming Wan   

          Actor_8        Actor_9    Director_0       Director_1  \
0            6253           3624      75907223          3060263   
1            None           None          None             None   
2            None           None          None             None   
3       Bret True  Ann Dalrymple  A. Dean Bell   Aaron Lubarsky   
4  Viva Tsvetnova  Monica Vicare    Éva Gárdos  Yoshiyuki Takei   

            Director_2      Director_3             Director_4       Director_5  
0               686693          137833                 106684            61667  
1                 None            None                   None             None  
2                 None            None                   None             None  
3           Alan Brown      Amos Gitai  Alejandro G. Iñárritu   Ben Sharpsteen  
4  Zulfah Otto-Sallies  William Morgan         Victor Saville  Youssef Chahine  

[5 rows x 31 columns]

In [50]:
final_df.write.save("/user/akarshsahu/data/Netflix_Data/processed_data_final.parquet", format = "parquet")

In [51]:
final_df.write.save("/user/akarshsahu/data/Netflix_Data/processed_data_final.csv", format = "csv")

In [ ]:
# Try reading the file
#df = spark.read.csv("/user/akarshsahu/data/Netflix_Data/processed_data.csv", inferSchema=True, header=True)